## Uploading files from your local file system

`files.upload` returns a dictionary of the files which were uploaded.
The dictionary is keyed by the file name and values are the data which were uploaded.
File 'data.csv' is uploaded from the desktop.

In [0]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


##Pandas Dataframe Creation
Storing the data in pandas dataframe df2 and then copied as df. Any further changes are made on df.

In [214]:
import pandas as pd
import io
df2 = pd.read_csv(io.BytesIO(uploaded['data.csv']))
df=df2.copy()
df

,clicks,complaints,hard_bounces,opens,rejects,sent,soft_bounces,template,time,unique_clicks,unique_opens,unsubs
0,0,1,0,0,0,0,0,NEW COLLECTION - MAY 2018,5/23/2018 0:00,0,0,0
1,0,0,0,0,0,0,1,NEW COLLECTION - MAY 2018,5/21/2018 1:00,0,0,0
2,0,0,0,0,0,0,1,NEW COLLECTION - MAY 2018,5/21/2018 0:00,0,0,0
3,0,0,0,0,0,0,10,NEW COLLECTION - MAY 2018,5/20/2018 23:00,0,0,0
4,0,0,0,0,0,0,1,NEW COLLECTION - MAY 2018,5/20/2018 21:00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14744,245,1,19,1540,6,1808,8,WINTER SALE - JAN 2019,01-10-2019 18:00,179,726,0
14745,109,0,7,620,4,971,3,WINTER SALE - JAN 2019,01-10-2019 17:00,81,312,0
14746,7,0,0,9,0,5,0,WINTER SALE - JAN 2019,01-10-2019 16:00,2,4,0
14747,5,0,0,4,0,2,0,WINTER SALE - JAN 2019,01-09-2019 17:00,1,1,0


##Data Pre-processing:
Variable 'time' is converted into the appropriate format. Further, it is split into 'year' and 'month' variable.

In [0]:
df['Date']=pd.to_datetime(df['time'])
df['Year']=pd.DatetimeIndex(df['Date']).year
df['Month_Num']=pd.DatetimeIndex(df['Date']).month

In [216]:
df.sort_values(['Month_Num','Year'], ascending =[True, True] , inplace=True) 
import calendar
df['Month'] = df['Month_Num'].apply(lambda x: calendar.month_abbr[x])
df

,clicks,complaints,hard_bounces,opens,rejects,sent,soft_bounces,template,time,unique_clicks,unique_opens,unsubs,Date,Year,Month_Num,Month
14572,0,0,0,0,0,0,3,WINTER SALE - JAN 2016,1/24/2016 22:00,0,0,0,2016-01-24 22:00:00,2016,1,Jan
14573,0,0,0,0,0,0,1,WINTER SALE - JAN 2016,1/24/2016 21:00,0,0,0,2016-01-24 21:00:00,2016,1,Jan
14574,0,0,0,0,0,0,2,WINTER SALE - JAN 2016,1/24/2016 20:00,0,0,0,2016-01-24 20:00:00,2016,1,Jan
14575,0,0,0,0,0,0,1,WINTER SALE - JAN 2016,1/24/2016 17:00,0,0,0,2016-01-24 17:00:00,2016,1,Jan
14576,0,0,0,0,0,0,1,WINTER SALE - JAN 2016,1/23/2016 21:00,0,0,0,2016-01-23 21:00:00,2016,1,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13698,0,0,0,2,0,8,0,RB Performance - MONO S1 - FLDC,12-01-2019 14:00,0,2,0,2019-12-01 14:00:00,2019,12,Dec
13916,1,0,0,4,0,4,0,RB Performance - MONO S1 - FLDC - TEST,12-06-2019 20:00,1,2,0,2019-12-06 20:00:00,2019,12,Dec
13917,5,0,0,1,0,2,0,RB Performance - MONO S1 - FLDC - TEST,12-06-2019 19:00,1,1,0,2019-12-06 19:00:00,2019,12,Dec
13927,0,0,0,8,0,8,0,RB Performance - PROSPECTS S0,12/20/2019 21:00,0,4,0,2019-12-20 21:00:00,2019,12,Dec


##Calculations on pandas:
1. 'Sent' and 'Delivered' variables are calculated.
2. Further, a 'combined' column is created in the pandas df that is the concatination of 'template','Month' and 'Year' column values.
3. This is done so that the pandas df can be grouped by this combined column and we can do further calculations.
4. After the grouping is done by the month and year for each template type, further 'Delivery_rate','Open_rate' and 'Click_rate' are calculated.
5. The 'combined' column is split into columns: 'Template', 'Month' and 'Year'
6. A full pandas dataframes containing all templates for all years are then saved as "df3.csv".

In [0]:
df['Sent'] = df.apply(lambda row: row['complaints'] + row['hard_bounces']+ row['opens']+ row['rejects']+ row['sent']+ row['soft_bounces'], axis=1)
df['Delivered']=df.apply(lambda row: row['Sent']- (row['hard_bounces']+ row['soft_bounces']),axis=1)

In [0]:
df['combined']=df['template'].astype(str)+'_'+df['Month'].astype(str)+'_'+df['Year'].astype(str)

In [0]:
df_new=df.groupby(['combined'])['Sent','Delivered','opens','clicks'].sum().reset_index()

In [0]:
df_new[['Template','Month','Year']] = df_new.combined.str.split("_",expand=True) 

In [221]:
df_new['Delivery_rate']=df_new.apply(lambda row: row['Delivered']/ row['Sent'],axis=1)
df_new['Open_rate']=df_new.apply(lambda row: row['opens']/ row['Sent'],axis=1)
df_new['Click_rate']=df_new.apply(lambda row: row['clicks']/ row['Sent'],axis=1)
df_new

,combined,Sent,Delivered,opens,clicks,Template,Month,Year,Delivery_rate,Open_rate,Click_rate
0,NEW COLLECTION - MAY 2018_May_2018,28202,27710,8105,1140,NEW COLLECTION - MAY 2018,May,2018,0.982554,0.287391,0.040423
1,NEW COLLECTION - OCTOBER 2018_Nov_2018,5,0,0,0,NEW COLLECTION - OCTOBER 2018,Nov,2018,0.000000,0.000000,0.000000
2,NEW COLLECTION - OCTOBER 2018_Oct_2018,45754,45419,15657,2220,NEW COLLECTION - OCTOBER 2018,Oct,2018,0.992678,0.342200,0.048520
3,NJ-WAREHOUSE-NOV2018_Nov_2018,2608,2597,1029,99,NJ-WAREHOUSE-NOV2018,Nov,2018,0.995782,0.394555,0.037960
4,NJ-WAREHOUSE-NOV2018_Oct_2018,13,13,8,1,NJ-WAREHOUSE-NOV2018,Oct,2018,1.000000,0.615385,0.076923
...,...,...,...,...,...,...,...,...,...,...,...
822,WINTER SALE - JAN 2019_Dec_2018,11,11,7,7,WINTER SALE - JAN 2019,Dec,2018,1.000000,0.636364,0.636364
823,WINTER SALE - JAN 2019_Jan_2019,79081,78061,27362,3770,WINTER SALE - JAN 2019,Jan,2019,0.987102,0.346000,0.047673
824,WINTER SALE 2018_Jan_2018,45020,44418,16061,2252,WINTER SALE 2018,Jan,2018,0.986628,0.356753,0.050022
825,rb-onboarding-e1-cali-test_Jul_2019,41,41,1,0,rb-onboarding-e1-cali-test,Jul,2019,1.000000,0.024390,0.000000


In [0]:
df3=df_new.loc[:,['Month','Year','Template','Sent','Delivered','Delivery_rate','Open_rate','Click_rate']]
from google.colab import files
df3.to_csv(r'df3.csv', index = False, header=True)

##value_counts():
This method is used on the 'Template' field to look into the unique values of the template and their number of occurence.

In [192]:
dfc=df3.Template.value_counts()
dfc

RB Onboarding E3-June-2016                       35
RB Onboarding E4 DCFL                            32
RB Performance - INACTIVES S4 - June 2016        30
RB Onboarding E4 CALI                            30
RB Performance - MONO S1                         29
                                                 ..
RB Performance - COLLECTORS S3 - CAN - FR         1
WINTER SALE - JAN 2017                            1
RB Performance - INACTIVES - S4 - CALI - TEST     1
PICTO-REACTIVATION-PROSPECTS-JAN25-2019           1
RB Performance - PROSPECTS S0 - CAN - FRA         1
Name: Template, Length: 86, dtype: int64

#Createing Report Dynamically:
Program asks the user to enter the templated for which it need the report. After entering one or more template numbers and their names, user is able to create a csv file report and download it in the local desktop.

In [223]:
mylist=[]
n=int(input('How many templates?\n'))
for i in range(n):
  value_for_filter = input('Enter what template would you like to look data at:\n')
  mylist.append(value_for_filter) # add input to the list

print(mylist)

How many templates?
2
Enter what template would you like to look data at:
RB Onboarding E4 DCFL
Enter what template would you like to look data at:
RB Onboarding E4 CALI
['RB Onboarding E4 DCFL', 'RB Onboarding E4 CALI']


In [224]:
df4 = df3[df3['Template'].isin(mylist)]
df4=df4.reset_index(drop=True)
df4

,Month,Year,Template,Sent,Delivered,Delivery_rate,Open_rate,Click_rate
0,Apr,2018,RB Onboarding E4 CALI,215,214,0.995349,0.502326,0.027907
1,Apr,2019,RB Onboarding E4 CALI,506,498,0.984190,0.640316,0.045455
2,Aug,2017,RB Onboarding E4 CALI,74,74,1.000000,0.121622,0.013514
3,Aug,2018,RB Onboarding E4 CALI,451,447,0.991131,0.580931,0.022173
4,Aug,2019,RB Onboarding E4 CALI,538,527,0.979554,0.462825,0.031599
...,...,...,...,...,...,...,...,...
57,Oct,2018,RB Onboarding E4 DCFL,182,182,1.000000,0.472527,0.065934
58,Oct,2019,RB Onboarding E4 DCFL,287,285,0.993031,0.599303,0.041812
59,Sep,2017,RB Onboarding E4 DCFL,21,21,1.000000,0.000000,0.000000
60,Sep,2018,RB Onboarding E4 DCFL,291,287,0.986254,0.542955,0.027491


In [0]:
from google.colab import files
df4.to_csv(r'df4.csv', index = False, header=True)
